In [3]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import re
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
import shutil


In [7]:
class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0
        self.input_folder = None  # Added line

    def create_output_folder(self):
        folder_number = 1
        while True:
            current_folder = f'ForClassification/Video_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def select_input_folder(self):
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        self.input_folder = filedialog.askdirectory(title="Select Video Folder")
        if not self.input_folder:
            print("No folder selected. Exiting.")
            self.input_folder = self.output_folder
        else:
            print(f"Selected folder: {self.input_folder}")

    
    def record_video(output_path='RecordedVids/output_video.mp4', duration=10):
        cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera

        # Set the video resolution to 720p
        cap.set(3, 1280)  # Width
        cap.set(4, 720)   # Height

        # Set the video codec and create a VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = 21  # Frames per second
        width, height = int(cap.get(3)), int(cap.get(4))
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        # Display the countdown
        for i in range(5, 0, -1):
            print(f"Recording starts in {i} seconds...")
            time.sleep(1)

        start_time = time.time()

        while time.time() - start_time < duration:
            ret, frame = cap.read()

            if not ret:
                print("Error capturing frame.")
                break

            # Display the frame (optional)
            cv2.imshow('Recording', frame)

            # Write the frame to the video file
            out.write(frame)

            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        print("Recording complete!")

        # Release the VideoCapture and VideoWriter objects
        cap.release()
        out.release()

        # Close the OpenCV windows
    cv2.destroyAllWindows()

    def classify_vid(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return
        # Start timing the classification process
        start_time = time.time()

        # adjust tkinter window to center
        progress_window = tk.Toplevel()
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while self.frame_count < 210:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            if self.frame_count % 10 == 0:
                elapsed_time = time.time() - start_time
                fps = self.frame_count / elapsed_time
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        elapsed_time = time.time() - start_time
        avg_fps = self.frame_count / elapsed_time
        avg_processing_time = elapsed_time / self.frame_count if self.frame_count > 0 else 0

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        progress_window.destroy()

        # Load the model
        model_input_size = (224, 224, 3)
        seed = 8
        batch_size = 16  # Change this shit pag sasabog na yung GPU
        exercise_type = ['LumbarSideBends', 'QuadrupedThoracicRotation', 'SupineNeckLift']

        model = load_model("DemoModelFinal1.h5")

        inference_generator = ImageDataGenerator(
            preprocessing_function=preprocess_input,
        )

        curr_folder = self.output_folder

        inference_direc = os.path.dirname(curr_folder)  # Get the parent directory of self.input_folder
        inference_direc = os.path.join(inference_direc, '')  # Add an additional '/' at the end
        inference_direc = inference_direc.replace("\\", "/")  # Replace backslashes with forward slashes

        print(f"Using optical flow folder: {inference_direc}")

        # Print the contents of the input folder
        print(f"Contents of input folder: {os.listdir(inference_direc)}")

        inference_data = inference_generator.flow_from_directory(
            inference_direc,
            target_size=model_input_size[:2],
            batch_size=batch_size,
            shuffle=False,
        )

        prediction = model.predict(inference_data)
        results = np.argmax(prediction, axis=1)
        predicted_label = [exercise_type[i] for i in results]

        def calculate_confidence_level(counts, confidence_level=95):
            total = sum(counts)
            max_index = counts.argmax()
            part = counts[max_index]

            percentage = (part / total) * 100

            # Calculate standard error and margin of error for the confidence interval
            standard_error = np.sqrt((percentage * (100 - percentage)) / total)
            margin_of_error = standard_error * (1.96)  # Z-score for 95% confidence interval

            lower_bound = max(0, percentage - margin_of_error)
            upper_bound = min(100, percentage + margin_of_error)

            return lower_bound, upper_bound

        unique_values, counts = np.unique(results, return_counts=True)

        for value, count in zip(unique_values, counts):
            print(f"{value}: {count} times")

        print("Counts list:", counts)  # Print the items in counts[]

        max_index = counts.argmax()
        max_value = unique_values[max_index]

        # Insert spaces before capital letters using a regular expression
        spaced_exercise_type = [re.sub('([A-Z])', r' \1', word) for word in exercise_type]

        lower_bound, upper_bound = calculate_confidence_level(counts)
        print(f"The exercise type is: {spaced_exercise_type[max_value]} with {lower_bound:.2f}% - {upper_bound:.2f}% Confidence Level")
        cl = upper_bound
        acc = lower_bound
        
        print(f"Average Processing Time: {avg_processing_time:.2f}s, Average FPS: {avg_fps:.2f}")

        self.results(self.video_path,spaced_exercise_type[max_value],acc,cl)
              
        # Delete the ForClassification folder
        shutil.rmtree(curr_folder)
        
    def track_features(self):
        self.get_video_path
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def results(self, video_path, exercise_type, acc, confidence_level):
        # Update the video path
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path)

        if not self.cap.isOpened():
            print(f"Error opening video file: {self.video_path}")
            return

        ret, frame = self.cap.read()
        if not ret or frame is None:
            print("Error reading frame.")
            return

        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            # Add text to the image
            text = f"The exercise type is: {exercise_type} at {acc:.2f}% with {confidence_level:.2f}% Confidence Level"
            position, font, font_scale, font_color, thickness = (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2

            cv2.putText(flow_image, text, position, font, font_scale, font_color, thickness)
            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()



    def show_optical_flow_video(self):
            self.get_video_path()
            self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def record_video():
        feature_tracker.record_video()
    
    def classify_exercise():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.classify_vid(progress_var)

    def show_optical_flow():
        feature_tracker.show_optical_flow_video()

    def exit_application():
        if root:
            root.destroy()

    btn_record_video = tk.Button(root, text="Record Exercise", command=record_video)
    btn_record_video.pack(pady=10)

    btn_classify_exercise = tk.Button(root, text="Classify Exercise", command=classify_exercise)
    btn_classify_exercise.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow)
    btn_show_optical_flow.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)


    root.mainloop()

Using optical flow folder: ForClassification/
Contents of input folder: ['Video_001']
Found 210 images belonging to 1 classes.
14/14 [==============================] - 47s 3s/step
0: 200 times
1: 6 times
2: 4 times
Counts list: [200   6   4]
The exercise type is:  Lumbar Side Bends with 92.36% - 98.12% Confidence Level
Processing time: 83.64 seconds
Processing speed: 0.00 f
